# Step 1: Load the Dataset {.unnumbered}

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, regexp_replace, transform, isnan


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


your 131072x1 screen size is bogus. expect trouble


25/04/17 20:02:13 WARN Utils: Your hostname, DESKTOP-AEE21PF resolves to a loopback address: 127.0.1.1; using 192.168.167.208 instead (on interface eth0)
25/04/17 20:02:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/17 20:02:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: timestamp (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: timestamp (nullable = true)
 |-- EXPIRED: timestamp (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: timestamp (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVEL

# Data Cleaning and Typecasting

In [2]:
# 1.DURATION = EXPIRED - POSTED

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

from pyspark.sql.functions import datediff, when, to_date, col

# 将日期列转换为日期格式
df = df.withColumn("POSTED", to_date("POSTED", "MM/dd/yyyy")) \
       .withColumn("EXPIRED", to_date("EXPIRED", "MM/dd/yyyy"))

# 直接用日期差填充 DURATION
df = df.withColumn(
    "DURATION",
    when(col("DURATION").isNull(), datediff("EXPIRED", "POSTED"))
    .otherwise(col("DURATION"))
)

df.select("POSTED", "EXPIRED", "DURATION").show(truncate=False)

+----------+----------+--------+
|POSTED    |EXPIRED   |DURATION|
+----------+----------+--------+
|2024-06-02|2024-06-08|6       |
|2024-06-02|2024-08-01|60      |
|2024-06-02|2024-07-07|35      |
|2024-06-02|2024-07-20|48      |
|2024-06-02|2024-06-17|15      |
|2024-06-02|2024-06-12|10      |
|2024-06-02|2024-08-01|60      |
|2024-06-02|2024-07-07|35      |
|2024-06-02|2024-06-20|18      |
|2024-06-02|2024-08-01|60      |
|2024-06-02|2024-09-27|117     |
|2024-06-02|2024-06-08|6       |
|2024-06-02|2024-07-05|33      |
|2024-06-02|2024-07-27|55      |
|2024-06-02|2024-08-01|60      |
|2024-06-02|2024-08-09|68      |
|2024-06-02|2024-06-20|18      |
|2024-06-02|2024-08-01|60      |
|2024-06-02|2024-08-01|60      |
|2024-06-02|2024-08-11|70      |
+----------+----------+--------+
only showing top 20 rows



In [4]:
# 2.SOURCE_TYPES, SOURCES, URL等列只保留双引号内容

from pyspark.sql.functions import regexp_extract, col
from functools import reduce

# 指定要处理的列
columns_to_clean = ["SOURCE_TYPES", "SOURCES", "URL", "EDUCATION_LEVELS_NAME", "SKILLS", "SKILLS_NAME", "SPECIALIZED_SKILLS", 
                    "SPECIALIZED_SKILLS_NAME", "CERTIFICATIONS", "CERTIFICATIONS_NAME", "COMMON_SKILLS", "COMMON_SKILLS_NAME", 
                    "SOFTWARE_SKILLS", "SOFTWARE_SKILLS_NAME", "CIP6", "CIP6_NAME", "CIP4", "CIP4_NAME", "CIP2", "CIP2_NAME", 
                    "LIGHTCAST_SECTORS", "LIGHTCAST_SECTORS_NAME"]  # 将列名替换为你要处理的列

# 批量清理
for col_name in columns_to_clean:
    df = df.withColumn(col_name, regexp_extract(col(col_name), r'"(.*?)"', 1))


df.show(10)

+--------------------+-------------------+----------------------+----------+----------+----------+--------+------------+------------------+--------------------+-----------+-------------------+--------------------+--------------------+-------------------+----------------+---------+--------------------+--------------------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+------------------+------+--------------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------

In [5]:
# 3.EDUCATION_LEVELS列只保留数字

from pyspark.sql.functions import regexp_extract

# 对 'EDUCATION_LEVELS' 列进行清理，只保留数字
df = df.withColumn("EDUCATION_LEVELS", regexp_extract("EDUCATION_LEVELS", r'(\d+)', 1))


df.select("EDUCATION_LEVELS").show(truncate=False)

+----------------+
|EDUCATION_LEVELS|
+----------------+
|2               |
|99              |
|2               |
|99              |
|99              |
|2               |
|2               |
|2               |
|99              |
|2               |
|0               |
|99              |
|2               |
|2               |
|99              |
|1               |
|1               |
|99              |
|2               |
|99              |
+----------------+
only showing top 20 rows



In [6]:
# 4.对LOCATION列进行处理

from pyspark.sql.functions import col, regexp_replace

# 去除大括号，并处理换行符和空格
df = df.withColumn("LOCATION", 
                           regexp_replace(regexp_replace(col("LOCATION"), r"\s*\n\s*", " "), r"[{}]", ""))


df.select("LOCATION").show(truncate=False)


+----------------------------------------+
|LOCATION                                |
+----------------------------------------+
| "lat": 33.20763, "lon": -92.6662674    |
| "lat": 44.3106241, "lon": -69.7794897  |
| "lat": 32.7766642, "lon": -96.7969879  |
| "lat": 33.4483771, "lon": -112.0740373 |
| "lat": 37.6392595, "lon": -120.9970014 |
| "lat": 0, "lon": 0                     |
| "lat": 33.4941704, "lon": -111.9260519 |
| "lat": 39.7589478, "lon": -84.1916069  |
| "lat": 41.1220409, "lon": -74.5804378  |
| "lat": 40.7501, "lon": -73.997         |
| "lat": 35.6224561, "lon": -117.6708966 |
| "lat": 21.3069444, "lon": -157.8583333 |
| "lat": 0, "lon": 0                     |
| "lat": 42.331427, "lon": -83.0457538   |
| "lat": 32.2987573, "lon": -90.1848103  |
| "lat": 42.3600825, "lon": -71.0588801  |
| "lat": 0, "lon": 0                     |
| "lat": 58.3019444, "lon": -134.4197221 |
| "lat": 33.5185892, "lon": -86.8103567  |
| "lat": 37.7749295, "lon": -122.4194155 |
+----------

In [7]:
# 5.MODELED_DURATION = MODELED_EXPIRED - POSTED

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

from pyspark.sql.functions import datediff, when, to_date, col

# 将日期列转换为日期格式
df = df.withColumn("MODELED_EXPIRED", to_date("MODELED_EXPIRED", "MM/dd/yyyy"))

# 直接用日期差填充 DURATION
df = df.withColumn(
    "MODELED_DURATION",
    when(col("MODELED_DURATION").isNull(), datediff("MODELED_EXPIRED", "POSTED"))
    .otherwise(col("MODELED_DURATION"))
)

df.select("POSTED", "MODELED_EXPIRED", "MODELED_DURATION").show(truncate=False)

+----------+---------------+----------------+
|POSTED    |MODELED_EXPIRED|MODELED_DURATION|
+----------+---------------+----------------+
|2024-06-02|2024-06-08     |6               |
|2024-06-02|2024-08-01     |60              |
|2024-06-02|2024-06-10     |8               |
|2024-06-02|2024-06-12     |10              |
|2024-06-02|2024-06-17     |15              |
|2024-06-02|2024-06-12     |10              |
|2024-06-02|2024-06-22     |20              |
|2024-06-02|2024-06-10     |8               |
|2024-06-02|2024-06-20     |18              |
|2024-06-02|2024-08-01     |60              |
|2024-06-02|2024-07-13     |41              |
|2024-06-02|2024-06-08     |6               |
|2024-06-02|2024-07-05     |33              |
|2024-06-02|2024-07-27     |55              |
|2024-06-02|2024-06-14     |12              |
|2024-06-02|2024-06-08     |6               |
|2024-06-02|2024-06-20     |18              |
|2024-06-02|2024-07-10     |38              |
|2024-06-02|2024-08-01     |60    

In [9]:
# 保存数据：
# 1. 使用 coalesce(1) 将所有分区的数据合并为一个文件
df.coalesce(1).write.option("header", "true").csv("data/lightcast_cleaned_temp")

# 2. 查找生成的文件并重命名
import os
import shutil

# 获取生成的文件路径
generated_file_path = 'data/lightcast_cleaned_temp'

# 获取文件夹中的所有文件（应只有一个文件）
for filename in os.listdir(generated_file_path):
    if filename.startswith('part-'):  # 找到 part 文件
        # 重命名并移动到目标位置
        shutil.move(os.path.join(generated_file_path, filename), 'data/lightcast_cleaned.csv')

# 删除临时文件夹
shutil.rmtree(generated_file_path)


# Load New Dataset

In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, regexp_replace, transform, isnan

spark = SparkSession.builder.appName("LightcastCleanedData").getOrCreate()

# 重新加载处理后的数据
df_cleaned = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").csv("data/lightcast_cleaned.csv")

# 查看数据结构和样本
df_cleaned.show()

your 131072x1 screen size is bogus. expect trouble


25/04/17 20:04:16 WARN Utils: Your hostname, DESKTOP-AEE21PF resolves to a loopback address: 127.0.1.1; using 192.168.167.208 instead (on interface eth0)
25/04/17 20:04:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/17 20:04:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/04/17 20:04:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+-------------------+----------------------+----------+-------------------+-------------------+--------+------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+-------------------+----------------+---------+--------------------+--------------------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+------------------+------+--------------------+-----+--------------------+-----+-------------+---------------+--------------------+-------